In [115]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import csv # implements classes to read and write tabular data in CSV form

print('Libraries imported.')

Libraries imported.


In [116]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [117]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    

In [118]:
nb_data = newyork_data['features']

In [119]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
nbs = pd.DataFrame(columns=column_names)

In [120]:
for data in nb_data:
    borough = nb_name = data['properties']['borough'] 
    
    nb_name = data['properties']['name']    
    nb_lat_long = data['geometry']['coordinates']
    nb_lat = nb_lat_long[1]
    nb_long = nb_lat_long[0]
    nbs = nbs.append({'Borough': borough,
                                          'Neighborhood': nb_name,
                                          'Latitude': nb_lat,
                                          'Longitude': nb_long}, ignore_index=True)

In [121]:
print('The dataframe has {} boroughs and {} neighborhouds.'.format(
        len(nbs['Borough'].unique()),
        nbs.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhouds.


In [122]:
nbs.to_csv('pop1.csv',index=False)
nbs.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [123]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent="Jupyter")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


print('Geo coordinates of NYC are {}, {}.'.format(latitude, longitude))

Geo coordinates of NYC are 40.7127281, -74.0060152.


In [133]:

# create map of Toronto using latitude and longitude values
map_NewYork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(nbs['Latitude'], nbs['Longitude'], nbs['Borough'], nbs['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5, popup=label,
        color='blue', fill_color='#2134ba',
        fill_opacity=0.6, parse_html=False).add_to(map_NewYork)  
    
map_NewYork

In [100]:
import matplotlib.pyplot as plt
# conda install -c anaconda beautiful-soup --yes
from bs4 import BeautifulSoup # package for parsing HTML and XML documents
print('Libraries imported.')

Libraries imported.


In [101]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City').text
soupy = BeautifulSoup(website_url,'lxml')
table = soupy.find('table',{'class':'wikitable sortable'})
headers = [header.text for header in table.find_all('th')]
table_rows = table.find_all('tr')        

rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)



with open('pop1a.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

In [102]:
Pop_1a=pd.read_csv('pop1a.csv')
Pop_1a.drop(Pop_1a.columns[[7,8,9,10,11]], axis=1,inplace=True)

Pop_1a.columns = Pop_data.columns.str.replace(' ', '')
Pop_1a.columns = Pop_data.columns.str.replace('\'','')
Pop_1a.rename(columns={'Borough':'persons_sq_mi','County':'persons_sq_km'}, inplace=True)
Pop_1a

,New York Citys five boroughsvte,Jurisdiction,Population,Gross Domestic Product,Land area,Density,persons_sq_mi,squarekm,persons / sq. mi,persons /sq. km
0,The Bronx\n,\n Bronx\n,"1,471,160\n",28.787\n,"19,570\n",42.10\n,109.04\n,NaN,NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,648,771\n",63.303\n,"23,900\n",70.82\n,183.42\n,NaN,NaN,NaN
2,Manhattan\n,\n New York\n,"1,664,727\n",629.682\n,"378,250\n",22.83\n,59.13\n,NaN,NaN,NaN
3,Queens\n,\n Queens\n,"2,358,582\n",73.842\n,"31,310\n",108.53\n,281.09\n,NaN,NaN,NaN
4,Staten Island\n,\n Richmond\n,"479,458\n",11.249\n,"23,460\n",58.37\n,151.18\n,NaN,NaN,NaN
5,City of New York,"8,622,698",806.863,"93,574",302.64,783.83,"28,188",NaN,NaN,NaN
6,State of New York,"19,849,399","1,547.116","78,354","47,214","122,284",416.4,NaN,NaN,NaN
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
Pop_1a.rename(columns = {'New York Citys five boroughsvte\n' : 'Borough',
                   'Jurisdiction\n':'County',
                   'Population\n':'Estimate_2017', 
                   'Land area\n':'square_miles',
                    'Density\n':'square_km'}, inplace=True)



Pop_1a

,Borough,County,Estimate_2017,Gross Domestic Product,square_miles,square_km,persons_sq_mi,squarekm,persons / sq. mi,persons /sq. km
0,The Bronx\n,\n Bronx\n,"1,471,160\n",28.787\n,"19,570\n",42.10\n,109.04\n,NaN,NaN,NaN
1,Brooklyn\n,\n Kings\n,"2,648,771\n",63.303\n,"23,900\n",70.82\n,183.42\n,NaN,NaN,NaN
2,Manhattan\n,\n New York\n,"1,664,727\n",629.682\n,"378,250\n",22.83\n,59.13\n,NaN,NaN,NaN
3,Queens\n,\n Queens\n,"2,358,582\n",73.842\n,"31,310\n",108.53\n,281.09\n,NaN,NaN,NaN
4,Staten Island\n,\n Richmond\n,"479,458\n",11.249\n,"23,460\n",58.37\n,151.18\n,NaN,NaN,NaN
5,City of New York,"8,622,698",806.863,"93,574",302.64,783.83,"28,188",NaN,NaN,NaN
6,State of New York,"19,849,399","1,547.116","78,354","47,214","122,284",416.4,NaN,NaN,NaN
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
Pop_1a['Borough']=Pop_1a['Borough'].replace(to_replace='\n', value='', regex=True)
Pop_1a['County']=Pop_1a['County'].replace(to_replace='\n', value='', regex=True)
Pop_1a['Estimate_2017']=Pop_1a['Estimate_2017'].replace(to_replace='\n', value='', regex=True)
Pop_1a['square_miles']=Pop_1a['square_miles'].replace(to_replace='\n', value='', regex=True)
Pop_1a['square_km']=Pop_1a['square_km'].replace(to_replace='\n', value='', regex=True)

Pop_1a


,Borough,County,Estimate_2017,Gross Domestic Product,square_miles,square_km,persons_sq_mi,squarekm,persons / sq. mi,persons /sq. km
0,The Bronx,Bronx,"1,471,160",28.787\n,"19,570",42.10,109.04\n,NaN,NaN,NaN
1,Brooklyn,Kings,"2,648,771",63.303\n,"23,900",70.82,183.42\n,NaN,NaN,NaN
2,Manhattan,New York,"1,664,727",629.682\n,"378,250",22.83,59.13\n,NaN,NaN,NaN
3,Queens,Queens,"2,358,582",73.842\n,"31,310",108.53,281.09\n,NaN,NaN,NaN
4,Staten Island,Richmond,"479,458",11.249\n,"23,460",58.37,151.18\n,NaN,NaN,NaN
5,City of New York,"8,622,698",806.863,"93,574",302.64,783.83,"28,188",NaN,NaN,NaN
6,State of New York,"19,849,399","1,547.116","78,354","47,214","122,284",416.4,NaN,NaN,NaN
7,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:

Pop_1a


,Borough,County,Estimate_2017,Gross Domestic Product,square_miles,square_km,persons_sq_mi,squarekm,persons / sq. mi,persons /sq. km
0,The Bronx,Bronx,"1,471,160",28.787\n,"19,570",42.10,109.04\n,NaN,NaN,NaN
1,Brooklyn,Kings,"2,648,771",63.303\n,"23,900",70.82,183.42\n,NaN,NaN,NaN
2,Manhattan,New York,"1,664,727",629.682\n,"378,250",22.83,59.13\n,NaN,NaN,NaN
3,Queens,Queens,"2,358,582",73.842\n,"31,310",108.53,281.09\n,NaN,NaN,NaN
4,Staten Island,Richmond,"479,458",11.249\n,"23,460",58.37,151.18\n,NaN,NaN,NaN
5,City of New York,"8,622,698",806.863,"93,574",302.64,783.83,"28,188",NaN,NaN,NaN
6,State of New York,"19,849,399","1,547.116","78,354","47,214","122,284",416.4,NaN,NaN,NaN
7,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
Pop_1a

Pop_1a.to_csv('pop1b.csv',index=False)

In [107]:
website_url = requests.get('https://en.wikipedia.org/wiki/New_York_City').text
soupy = BeautifulSoup(website_url,'lxml')
table = soupy.find('table',{'class':'wikitable sortable collapsible'})
headers = [header.text for header in table.find_all('th')]
table_rows = table.find_all('tr')        

rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('NYC_DEMO.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

In [108]:
data_z=pd.read_csv('NYC_DEMO.csv')
data_z.rename(columns = {'Racial composition' : 'Composition', 
                         '2010[249]' : '2010',
                         '1990[251]':'1990',
                         '1970[251]':'1970', 
                         '1940[251]\n':'1940',
                        }, inplace=True)
data_z['1970'] = data_z['1970'].str.rstrip('[252]')

data_z.columns = data_z.columns.str.replace(' ', '')
data_z= data_z.replace('\n','', regex=True)
data_z

,Composition,2010,1990,1970,1940
0,White,44.0%,52.3%,76.6%,93.6%
1,—Non-Hispanic,33.3%,43.2%,62.9%,92.0%
2,Black or African American,25.5%,28.7%,21.1%,6.1%
3,Hispanic or Latino (of any race),28.6%,24.4%,16.2%,1.6%
4,Asian,12.7%,7.0%,1.2%,−


In [109]:
data_z.to_csv('pop2.csv',index=False)


In [110]:
print("The diversity of NYC could be mainly classified into these types :") 
data_z['Composition']

The diversity of NYC could be mainly classified into these types :


0                               White
1                       —Non-Hispanic
2           Black or African American
3    Hispanic or Latino (of any race)
4                               Asian
Name: Composition, dtype: object

### It is also important to note that there is a steady increase in the population of the Hispanic/Latino and Asians in these regions. This would give us a deeper insight in changing trends for the years to come.